1. https://www.bilibili.com/video/BV18g4119737?p=21&vd_source=70200f7d09862fd682e5f89b22c89125
2. 涉及到4组维度变换的函数
    - view, reshape
    - squeeze, unsqueeze
    - transpose,t, permute
    - expand, repeat

# 0. Preparations

In [1]:
import torch

# 1. view, reshape
## 1.1 view及其物理意义
1. `a = torch.rand(4,1,28,28)`代表的是batchsize=4,channel=1, pic shape = 28*28
    - B*C*H*W
2. 重点是关注物理意义/ 对数据的理解
    - 容易造成数据污染,因为`view`了之后<u>没办法还原</u>数据本来的通道或者大小

In [2]:
a = torch.rand(4,1,28,28)
print("a.shape: ",a.shape)

a.shape:  torch.Size([4, 1, 28, 28])


3. 这个适合全连接层,把后面除了batch的信息拉成一个向量

In [3]:
print("a.shape after a.view(4,28*28): ",a.view(4,28*28).shape)


a.shape after a.view(4,28*28):  torch.Size([4, 784])


4. 这个时候focus on “行”的信息,把每行信息抽取出来,这么多batch这么多channel中一共有`4*28`行

In [4]:
print("a.shape after a.view(4*28,28): ",a.view(4*28,28).shape)

a.shape after a.view(4*28,28):  torch.Size([112, 28])


5. 这个时候值focus on 每一个pic的每一层信息,不管是哪个batch的哪个channel的,只关注2维的“方框框”

In [5]:
print("a.shape after a.view(4*1,28,28): ",a.view(4*1,28,28).shape)

a.shape after a.view(4*1,28,28):  torch.Size([4, 28, 28])


# 2. unsqueeze和维度理解
## 2.1 添加维度的位置如何理解
1. 下面两个unsqueeze的位置不一样
2. 当添加在最后一个dim时,增加的是**低一级**的维度,shape 从[2]变成[2,1],新增的维度是对每一个高维维度同时有效的,所以[]把1.2和2.3**分别**括起来了
3. 而当添加在第一个dim时,增加的是**高一级**的维度,shape 从[2]变成[1,2],新增的维度是对所有低维维度同时有效的,所以[]把1.2和2.3**同时**括起来了

In [6]:
a = torch.tensor([1.2,2.3])
print("a.shape: ",a.shape)

a1 = a.unsqueeze(-1)
print("\na.unsqueeze(-1): \n")
print(a1)
print(a1.shape)
a2 = a.unsqueeze(0)

print("\na.unsqueeze(0): \n")
print(a2)
print(a2.shape)

a.shape:  torch.Size([2])

a.unsqueeze(-1): 

tensor([[1.2000],
        [2.3000]])
torch.Size([2, 1])

a.unsqueeze(0): 

tensor([[1.2000, 2.3000]])
torch.Size([1, 2])


## 2.2 example
1. 现在要给一个图片`f`添加一个bias`b`,这个bias相当于要给每个channel上的所有像素增加一个偏置
2. 所以这个bias是加在`32`上的

In [7]:
# 显然这两个dim不同,要想办法把32放在最后的像素上
b = torch.rand(32)
f = torch.rand(4,32,14,14)
print("before unsqueeze: ",b.shape)

b = b.unsqueeze(1).unsqueeze(2).unsqueeze(0)
print("after unsqueeze: ",b.shape)

before unsqueeze:  torch.Size([32])
after unsqueeze:  torch.Size([1, 32, 1, 1])


# 3. expand, repeat
1. 经过unsqueeze后,`b=[1, 32, 1, 1]`,但是这个时候依然不能和`f`相加,因为shape不同
2. 两个最终效果一样的函数：
    - expand： broadcasting。**推荐**,节约内存,只在<u>必要的时候进行copy</u>
    - repeat：memory copied

## 3.1 expand
3. `b=[1, 32, 1, 1]`
    - 第一次expand之后,一些维度从`1`变成`4`或者`14`,这都是ok的。
    - 但是如果`b=[3, 32, 3, 3]`,就不能expand成`4`或者`14`,这个大小差距没有办法进行“自动”复制

In [8]:
a = torch.tensor([4,32,14,14])

print("b.shape: ",b.shape)

print("b.shape after expand: ",b.expand(4,32,14,14).shape)

# 偷懒
print("b.shape after expand: ",b.expand(-1,32,-1,-1).shape)

b.shape:  torch.Size([1, 32, 1, 1])
b.shape after expand:  torch.Size([4, 32, 14, 14])
b.shape after expand:  torch.Size([1, 32, 1, 1])


## 3.2 repeat
1. repeat函数中给的是<u>“在这个维度上要拷贝多少次”</u>,因此`32`被拷贝`32`次后,32*32=1024
2. 不建议使用,因为会占用新的内存空间

In [9]:
print("b.shape: ",b.shape)

print("b.shape after repeat: ",b.repeat(4,32,1,1).shape)

b.shape:  torch.Size([1, 32, 1, 1])
b.shape after repeat:  torch.Size([4, 1024, 1, 1])


# 4. transpose,t, permute
## 4.1 transpose以及view：进行维度跟踪
1. `transpose`给出要交换哪两个维度:这里是从a的BCHW 变成a1的BWHC

In [10]:
a = torch.rand(4,3,32,32)
print("a.shape: ",a.shape)

a1 = a.transpose(1,3)
print("a.shape after a.transpose(1,3): ",a1.shape)

a.shape:  torch.Size([4, 3, 32, 32])
a.shape after a.transpose(1,3):  torch.Size([4, 32, 32, 3])


2. 现在试图经过过transpose和view之后,还原a的信息
    - 问题一：view会损失信息, 从a1的BWHC经过view之后,无法直接还原到最初的a的BCHW,顺序会变
    - 你怎么保证还原回来是BCHW而不是BCWH呢

In [11]:
a1 = a1.view(4,3*32*32).view(4,3,32,32)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

3. 问题二：数据`transpose`交换之后会变成<u>非连续的,他们之间的相对位置会发生改变</u>,需要`contiguous`
4. 这里会发现,`a1`的shape和`a`的BCHW不一样：
    - a1的BWHC,到B(WHC),再把`3`提到前面变成BCWH, 这里WH的相对位置错了

In [14]:
a1 = a.transpose(1,3).contiguous().view(4,3*32*32).view(4,3,32,32)
print("a1.shape: ",a1.shape)

a1.shape:  torch.Size([4, 3, 32, 32])


5. 正确做法,一是`view`的时候并不把`3`先提到前面,二是在最后在`transpose`一下,把BWHC变成BCHW(和a一样)

In [15]:
a2 = a.transpose(1,3).contiguous().view(4,3*32*32).view(4,32,32,3).transpose(1,3)
print("a2.shape: ",a2.shape)

a2.shape:  torch.Size([4, 3, 32, 32])


## 4.2 如何检验两个tensor完全一样
1. 先`eq`后`all`,前者保证数据内容一致,后者是所有内容都相同返回`true`
    - 后者有`sum(True)==n`的作用

In [16]:
print("a1和a完全一样吗：",torch.all(torch.eq(a,a1)))
print("a2和a完全一样吗：",torch.all(torch.eq(a,a2)))

a1和a完全一样吗： tensor(False)
a2和a完全一样吗： tensor(True)


## 4.3 permute
1. numpy存储图像用的是BHWC的格式,C在最后,所以要转换成这个格式
2. `transpose`和`permute`都可以做到,后者更简单快捷罢了,后者传进去4个维度你希望它所在的各个位置
3. 依然会打乱内存,如果需要恢复,和`transpose`一样,需要`.contiguous()`

In [25]:
a = torch.rand(4,3,28,28)
print("a.shape: ",a.shape)
print("a.shape after a.transpose(1,3): ",a.transpose(1,3).shape)

b = torch.rand(4,3,28,32)
print("\nb.shape: ",b.shape)
print("b.shape after b.transpose(1,3): \n",b.transpose(1,3).shape)

print("b.shape after b.transpose(1,3).transpose(1,2): \n",b.transpose(1,3).transpose(1,2).shape)
print("Now b is BHWC\n")

print("b.shape after b.permute(0,2,3,1): \n",b.permute(0,2,3,1).shape)
print("Now b is also BHWC\n")

a.shape:  torch.Size([4, 3, 28, 28])
a.shape after a.transpose(1,3):  torch.Size([4, 28, 28, 3])

b.shape:  torch.Size([4, 3, 28, 32])
b.shape after b.transpose(1,3): 
 torch.Size([4, 32, 28, 3])
b.shape after b.transpose(1,3).transpose(1,2): 
 torch.Size([4, 28, 32, 3])
Now b is BHWC

b.shape after b.permute(0,2,3,1): 
 torch.Size([4, 28, 32, 3])
Now b is also BHWC

